In [1]:
import hopsworks

In [3]:
HOPSWORKS_PROJECT_NAME = 'taxi_demand_rs'

In [4]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [5]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 was already in local storage
File 2023-08 was already in local storage
File 2023-09 was already in local storage
File 2023-10 was already in local storage
File 2023-11 was already in local storage
File 2023-12 was already in local 

Now we have all rides from the start of 2022 until the end of July 2024. This is because the site we are getting data from has not uploaded any data past this date.

In [6]:
rides

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
...,...,...
3076898,2024-07-31 23:12:00,243
3076899,2024-07-31 23:10:34,170
3076900,2024-07-31 23:32:00,197
3076901,2024-07-31 23:32:52,230


We have 100 million rides now.

In [7]:
print(f'{len(rides)=:,}')

len(rides)=101,372,930


In [8]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

2024-10-08 11:16:59,947 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

2024-10-08 11:16:59,986 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



100%|██████████| 265/265 [00:06<00:00, 43.44it/s]


Now we have the rides per hour for each pickup_location.

In [9]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,0,1
1,2022-01-01 01:00:00,0,1
2,2022-01-01 02:00:00,0,1
3,2022-01-01 03:00:00,0,1
4,2022-01-01 04:00:00,1,1
...,...,...,...
5997475,2024-07-31 19:00:00,2,265
5997476,2024-07-31 20:00:00,1,265
5997477,2024-07-31 21:00:00,4,265
5997478,2024-07-31 22:00:00,2,265


In [13]:
# Convert string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# Add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype('int64') // 10**6

This operation added a **pickup_ts** column to help hopsworks with the event time primary key.

In [14]:
ts_data

,pickup_hour,rides,pickup_location_id,pickup_ts
0,2022-01-01 00:00:00+00:00,0,1,1640995200000
1,2022-01-01 01:00:00+00:00,0,1,1640998800000
2,2022-01-01 02:00:00+00:00,0,1,1641002400000
3,2022-01-01 03:00:00+00:00,0,1,1641006000000
4,2022-01-01 04:00:00+00:00,1,1,1641009600000
...,...,...,...,...
5997475,2024-07-31 19:00:00+00:00,2,265,1722452400000
5997476,2024-07-31 20:00:00+00:00,1,265,1722456000000
5997477,2024-07-31 21:00:00+00:00,4,265,1722459600000
5997478,2024-07-31 22:00:00+00:00,2,265,1722463200000


In [15]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1049751


In [18]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


Now we will write data to the feature group within our feature store.

In [17]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [19]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

In [20]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1049751/fs/1041478/fg/1260875


Uploading Dataframe: 0.00% |          | Rows 0/5997480 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1049751/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1fde8d59780>, None)